### Setup

- Generate cooccurrences (value represents number of articles in which two companies occur together)

TODO:
- Other measures for cooccurrence?

In [2]:
import os
import re
import glob
import itertools
import matplotlib.patheffects as path_effects
from datetime import datetime
import sys
sys.path.append("..") # Adds higher directory to python modules path for importing from src dir

import pandas as pd
import numpy as np
import tqdm
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from src.datasets import NyseStocksDataset, NyseSecuritiesDataset, NyseFundamentalsDataset
import src.econometric_utils as eco
import src.regression_utils as regr
import src.plot_utils as plot
import src.math_utils as math_utils
import src.nlp_utils as nlp_utils
import src.utils as utils

%matplotlib inline
%load_ext autotime
%load_ext autoreload
%autoreload 2

c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [3]:
stocks_ds = NyseStocksDataset(file_path='../data/nyse/prices-split-adjusted.csv')
securities_ds = NyseSecuritiesDataset(file_path='../data/nyse/securities.csv')
companies = securities_ds.get_all_company_names()  # List[Tuple[symbol, name]]
stocks_ds.load()


time: 28.3 s


### Preprocess & Validate

In [4]:
NEWS = os.path.join("..", "data", "preprocessed", "news-v3.csv")
articles = pd.read_csv(NEWS, index_col=0)
# articles = pd.read_csv(NEWS, index_col=0, nrows=people_occ.article_id.max()+50)
# articles = pd.read_csv(NEWS, index_col=0, skiprows=range(1, people_occ.article_id.max()))
# articles = articles[articles.index < 106519]

time: 48.8 s


In [ ]:
def find_increases(people_occ, article, add=0, strict=False):
    increases = {}
    pbar = tqdm(articles.index)
    for article_id in pbar:
        for i, row in people_occ[people_occ.article_id == article_id].iterrows():
            if pd.isna(articles.loc[article_id+add].content) or articles.loc[article_id+add].content[row.start_idx:row.end_idx] != row.match_text:
                if strict:
                    raise ValueError(f'No match for {i} with {row.match_text} in article {article_id}')
                j = 0
                while j < 600 and (pd.isna(articles.loc[article_id+add+j].content) or articles.loc[article_id+add+j].content[row.start_idx:row.end_idx] != row.match_text):
                    j += 1
                add += j
                assert articles.loc[article_id+add].content[row.start_idx:row.end_idx] == row.match_text, (article_id, i, j)
                increases[article_id] = add
                pbar.set_description(f'{article_id} -> {add}')
                # print(article_id, add)
        for i, row in comp_occ[comp_occ.article_id == article_id].iterrows():
            assert articles.loc[article_id].content[row.start_idx:row.end_idx] == row.match_text
    return increases

# x = find_increases(people_occ_bb, articles)

In [ ]:
for i, row in tqdm(people_occ.iterrows(), total=len(people_occ)):
    assert articles.loc[row.article_id].content[row.start_idx:row.end_idx] == row.match_text

In [ ]:
comp_names = securities_ds.get_all_company_names()
assert all([com in comp_names.Name.values for com in people_occ.company.unique()]), 'Sanity Check: Valid company names'
assert all(people_occ.notna().any()), 'Sanity Check: No missing values'

TODO: Validate correctness article_ids, start_idx and end_idx

TODO: Repeat after Bloomberg was added

In [ ]:
for x in tqdm(comp_occ.match_text.values):
    assert not securities_ds.get_most_similar_company(x, debug=True)[1], x

Regarding Bloomberg Company Occ from Anacode:


- Reuters ID: 0 to 106518
- Bloomberg: b0 equals 106519, b448394 equals 554913 (shift by 106519)

__Problems:__
- Article matching: 2 should be 106520, 447759 should be 554904
- The difference of ids shift increases by 626 -> counter appears to increase ID again and again (same as for reuters)
- 490 companies (I only have 443)
- Many false positives like 169 (Watson Pharam matches to WLTW in article 108(+106519)

###  Check for Parantheses

In [7]:
comp_occ = pd.read_csv('../data/preprocessed/occurrences/occurrences.csv', index_col=0).reset_index(drop=True)

time: 1.05 s


In [78]:
comp_occ['text_symbol'] = None
re_symbol = re.compile('^\s*\(\s*(\S*)\s*\)')

time: 229 ms


In [79]:
for i, row in tqdm(comp_occ.iterrows(), total=len(comp_occ)):
    following_str = articles.loc[row.article_id].content[row.end_idx:row.end_idx + 10]
    if len(following_str):
        match = regex.match(following_str)
        if match:
            comp_occ.loc[i, 'text_symbol'] = match.group(1)

time: 24min 25s


#### Manually fix some errors

In [264]:
hits = comp_occ[~comp_occ.text_symbol.isna()].copy()

hits.loc[20942, 'text_symbol'] = 'BAC'  # Error in text: Boeing symbol given eventhough it is about Bank of America
hits.loc[51716, 'text_symbol'] = 'AVGO'  # Error in text: BRC is brady and not Broadcom
hits.loc[59549, 'text_symbol'] = 'MCD'  # Error in text: MDC is Mizel Development Corporation and not McDonalds
hits.loc[59647, 'text_symbol'] = 'MCD'
hits.loc[63486, 'text_symbol'] = 'M'  # Error in text: GM is General Motors and not Macy's Inc

hits.text_symbol = hits.text_symbol.str.replace('APPL', 'AAPL')  # Apple
hits.text_symbol = hits.text_symbol.str.replace('AAMRQ', 'AAL')  # American Airlines
hits.text_symbol = hits.text_symbol.str.replace('BRCM', 'AVGO')  # Broadcom
hits.text_symbol = hits.text_symbol.str.replace('BFa\.N', 'BF.B')  # Brown-Forman Corp
hits.text_symbol = hits.text_symbol.str.replace('BFb\.N', 'BF.B')  # Brown-Forman Corp
hits.text_symbol = hits.text_symbol.str.replace('BRKa', 'BRK.B')  # Berkshire Hathaway Inc
hits.text_symbol = hits.text_symbol.str.replace('LTD', 'LB')  # L Brands Inc: https://finance.yahoo.com/news/l-brands-change-stock-symbol-230635555.html?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAADdWpaXzM_sbpZx35HoaXiwJeYKOkzAV8O-tnHCa8300RLd2kkmzJ44QDurWdTqG4xEsRu9yHbr9OF6e9gAwTUB65GqzIJoztbHNpo-2AFvokI3vO8TNVRAct4oAc4Q2bVchqpgcuPYNva1FPkO6S0IvZPwpQsVUeVZ9o5PYTQk0
hits.text_symbol = hits.text_symbol.str.replace('BRK\/A', 'BRK.B')  # Berkshire Hathaway Inc
hits.text_symbol = hits.text_symbol.str.replace('BRK\/B', 'BRK.B')  # Berkshire Hathaway Inc
hits.text_symbol = hits.text_symbol.str.replace('VIA\/B', 'VIAB')  # Viacom
hits.text_symbol = hits.text_symbol.str.replace('CBSa\.N', 'CBS')  # CBS Corp
hits.text_symbol = hits.text_symbol.str.replace('BofA', 'BAC')  # Bank of America
hits.text_symbol = hits.text_symbol.str.replace('P\&G', 'PG')  # Procter and Gamble
hits.text_symbol = hits.text_symbol.str.replace('UTC', 'UTX')  # United Technologies
hits.text_symbol = hits.text_symbol.str.replace('\.O', '')  # 424 hits, e.g. 'MSFT.O', 'NFLX.O'
hits.text_symbol = hits.text_symbol.str.replace('\.N', '')  # 28660 hits, e.g. 'XOM.N', 'WMT.N'
hits.text_symbol = hits.text_symbol.str.replace('\.L', '')  # 'CCL.L'
hits.text_symbol = hits.text_symbol.str.replace('\.C', '')  # 'HAL.C'
hits.loc[hits.text_symbol == 'TI', 'text_symbol'] = 'TXN'  # Texas Instruments Inc
hits.loc[hits.text_symbol == 'NWS', 'text_symbol'] = 'NWSA'  # News Corp
hits = hits[hits.text_symbol == hits.text_symbol.str.upper()]  # Country, etc
hits.text_symbol = hits.text_symbol.str.replace(r'[^\w\.]', '')
hits = hits[~hits.text_symbol.str.isdigit()]  # Year, Number
hits = hits[~hits.text_symbol.str.contains(r'\d')]
# hits.text_symbol = hits.text_symbol.str.replace(r'[^\w\.]', '')

time: 1.67 s


In [265]:
# hits = comp_occ[~comp_occ.text_symbol.isna()].copy()
# hits[hits.text_symbol.str.endswith('.O')]  # 424 hits
# hits[hits.text_symbol.str.endswith('.N')]  # 28660 hits
# hits[hits.text_symbol.str.endswith('.L')]
# hits[hits.text_symbol.str.endswith('.C')]  # Halliburton
# hits[hits.text_symbol != hits.text_symbol.str.upper()]
# hits[hits.text_symbol == 'TI']
hits[hits.stock_symbol != hits.text_symbol]

,article_id,stock_symbol,match_text,start_idx,end_idx,text_symbol
90153,42746,FTI,FMC Technologies Inc,2058,2078,FMC
92696,43896,AAL,American Airlines,1340,1357,AA
93061,44100,COP,ConocoPhillips,3488,3502,CVX
116794,57617,FIS,Fidelity National Information Services Inc,3056,3098,FNF
123402,60530,BAC,Bank of America Corp's,3111,3133,C
126824,62472,BA,Boeing Co,205,214,B
145880,72885,T,AT&T,443,447,AT
147916,74070,MS,Morgan Stanley,219,233,GS
152803,76514,KMI,Kinder Morgan Inc,3205,3222,KMP
157516,78458,PPG,PPG Industries,3575,3589,PPG.AX


time: 261 ms


#### Result:

In [278]:
acc = 100 * (1 - len(hits[hits.stock_symbol != hits.text_symbol]) / len(hits))
print(f'Accuracy: {acc:.4f}')

Accuracy: 99.8300
time: 257 ms


### Data

In [284]:
all_correlations = pd.read_csv('../data/preprocessed/econometrics/correlations.csv', index_col=[0, 1])
all_correlations.index.levels[1].name = None
all_correlations = all_correlations[[a < b for a, b in all_correlations.index]]
all_correlations.index = all_correlations.index.remove_unused_levels()

comp_occ = pd.read_csv('../data/preprocessed/occurrences/occurrences.csv', index_col=0).reset_index(drop=True)
# FIXME: Remove later
# comp_occ = comp_occ[comp_occ.article_id <= 106493]
comp_coocc = nlp_utils.get_cooccurrences(comp_occ).stack()
comp_coocc = comp_coocc[[a < b for a, b in comp_coocc.index]]
comp_coocc.index = comp_coocc.index.remove_unused_levels()

time: 27.9 s


In [83]:
# ', '.join(map(lambda x: f'{x[0]}: {x[1]}', x.items()))
increments_re = pd.Series({ 26944: 1, 55568: 2, 55624: 3, 56146: 4, 60323: 5, 68948: 6, 68978: 7, 72066: 8, 72779: 9, 74970: 10, 75163: 11, 75690: 12, 80466: 13, 81443: 14, 88575: 15, 89332: 16, 92712: 17, 94556: 18, 95271: 19, 95551: 20, 98957: 21, 99316: 22, 102077: 23, 102831: 24, 104671: 25 })
increments_bb = pd.Series({ 107371: 6, 107384: 12, 107401: 13, 107423: 16, 107437: 23, 107458: 27, 107487: 34, 107502: 36, 107522: 38, 107546: 43, 107563: 45, 107585: 50, 107628: 63, 107638: 69, 107644: 70, 107651: 76, 107657: 78, 107702: 94, 107707: 97, 107733: 106, 107779: 124, 107803: 130, 107845: 191, 107892: 271, 107930: 273, 107942: 275, 107955: 276, 107964: 278, 107978: 282, 107990: 286, 108046: 287, 108251: 288, 108528: 289, 108566: 290, 108633: 292, 108753: 293, 108874: 296, 109128: 297, 109261: 299, 109442: 300, 109536: 302, 109639: 306, 109708: 308, 109922: 309, 110074: 310, 110236: 311, 110446: 312, 110496: 313, 110624: 314, 110754: 315, 110898: 316, 111111: 318, 111293: 319, 111436: 321, 111494: 322, 111675: 324, 111731: 326, 111882: 327, 112465: 329, 112616: 330, 112647: 331, 113071: 332, 113176: 333, 113321: 335, 113766: 338, 113846: 339, 113850: 340, 113942: 341, 114207: 343, 114657: 346, 114932: 349, 114980: 351, 115126: 352, 115217: 353, 115336: 354, 115487: 356, 115704: 359, 115865: 360, 116048: 361, 116136: 362, 116510: 364, 116669: 369, 116681: 370, 116866: 372, 117270: 374, 117559: 375, 117687: 377, 118014: 379, 118808: 382, 119147: 384, 119236: 385, 119256: 386, 119688: 388, 119843: 389, 119969: 390, 120140: 392, 120292: 393, 120356: 394, 120712: 396, 120965: 399, 121265: 400, 121535: 401, 121684: 402, 121791: 404, 122103: 405, 122435: 407, 122654: 408, 122879: 409, 123177: 410, 123739: 411, 123770: 412, 124177: 415, 124317: 416, 124398: 417, 124570: 418, 124581: 419, 124669: 420, 124938: 421, 125002: 422, 125394: 423, 125508: 424, 125617: 426, 125771: 427, 125827: 428, 126015: 434, 126103: 435, 126588: 436, 126740: 437, 126879: 439, 127110: 442, 127322: 444, 127485: 445, 127825: 447, 127929: 448, 128181: 451, 128322: 454, 128379: 457, 128480: 458, 128798: 459, 129010: 462, 129084: 463, 129205: 464, 129391: 465, 129624: 469, 129875: 471, 129905: 472, 130211: 475, 130499: 477, 130623: 478, 130798: 479, 130833: 480, 131088: 485, 131110: 486, 131229: 487, 131515: 489, 148956: 490, 149498: 493, 149513: 494, 149517: 495, 149787: 508, 149856: 510, 150010: 511, 151114: 512, 151344: 513, 151462: 514, 151640: 515, 151964: 520, 152008: 521, 152159: 522, 152211: 523, 152417: 524, 163470: 525, 163653: 526, 163662: 528, 167219: 530, 167253: 533, 167292: 546, 167366: 554, 167408: 555, 293997: 566, 294038: 579, 294068: 587, 294114: 590, 416623: 592, 416691: 593, 416820: 595, 416865: 596, 474335: 597, 474439: 598, 474626: 599, 474676: 600, 474787: 603, 487231: 605, 487458: 607, 487571: 608, 487655: 609, 487659: 610, 487686: 611, 487723: 612, 487758: 614, 487784: 616, 487906: 622, 488022: 626 })

def get_increase(idx):
    increments = increments_re if idx < 106535 else increments_bb
    diffs = idx - increments.index
    if all(diffs < 0):
        return 0
    return increments.iloc[diffs[diffs >= 0].argmin()]

time: 250 ms


In [84]:
people_occ_re = pd.read_csv('../data/preprocessed/occurrences/occurrences-reuters-people-enriched.csv')
people_occ_re = people_occ_re[people_occ_re.columns[2:]]
people_occ_re.article_id += people_occ_re.article_id.apply(get_increase)

people_occ_bb = pd.read_csv('../data/preprocessed/occurrences/occurrences-bloomberg-people-enriched.csv', index_col=0)
people_occ_bb.article_id += 106519
people_occ_bb.article_id += people_occ_bb.article_id.apply(get_increase)

people_occ = pd.concat([people_occ_re, people_occ_bb])
people_occ.reset_index(drop=True, inplace=True)

name2symbol = dict([(com, securities_ds.get_company(com)) for com in people_occ.company.unique()])
people_occ['stock_symbol'] = people_occ.company.apply(name2symbol.get)
people_coocc = nlp_utils.get_cooccurrences(people_occ).stack()
people_coocc = people_coocc[[a < b for a, b in people_coocc.index]]

time: 13.3 s


Number of people occurrences: 19,009 (7,538 in Reuters, 11,471 in Bloomberg)
Number of company occurrences: 463,595

In [6]:
columns = ['article_id', 'start_idx', 'end_idx', 'stock_symbol']
mixed_occ = pd.concat([comp_occ[columns], people_occ[columns]]).sort_values(['article_id', 'start_idx']).reset_index(drop=True)
mixed_occ.drop_duplicates(['article_id', 'start_idx', 'end_idx'], inplace=True)
mixed_occ.reset_index(drop=True, inplace=True)

time: 865 ms


#### Only during train set

In [ ]:
TRAIN_FRAME = pd.to_datetime('2010-01-05'), pd.to_datetime('2012-05-21')
VAL_FRAME = pd.to_datetime('2012-05-22'), pd.to_datetime('2012-12-31')
NEWS = os.path.join("..", "data", "preprocessed", "news-v3.csv")
# articles = pd.read_csv(NEWS, index_col=0, nrows=people_occ.article_id.max()+50,)
articles_dates = pd.read_csv(NEWS, index_col=0, usecols=[0, 1, 2])
articles_dates.date = pd.to_datetime(articles_dates.date)
article_ids_before_train = set(articles_dates.index[articles_dates.date < TRAIN_FRAME[0]])

In [ ]:
# comp_occ_train = comp_occ[~comp_occ.article_id.isin(article_ids_before_train)].reset_index(drop=True)
# comp_coocc_train = nlp_utils.get_cooccurrences(comp_occ_train).stack()
# comp_coocc_train = comp_coocc_train[[a < b for a, b in comp_coocc_train.index]]

# people_occ_train = people_occ[~people_occ.article_id.isin(article_ids_before_train)]
# people_coocc_train = nlp_utils.get_cooccurrences(people_occ_train).stack()
# people_coocc_train = people_coocc_train[[a < b for a, b in people_coocc_train.index]]

- 40.702.147 Entities
- 9.636.490 tagged as ORG
- 463.595 linked to my stock companys (distributed over 126.583 articles)

Damerau-Levenshtein distance is not working (too repelling)

### Inspect Articles For Show Cases

In [11]:
NEWS = os.path.join("..", "data", "preprocessed", "news-v3.csv")
# articles = pd.read_csv(NEWS, index_col=0, nrows=people_occ.article_id.max()+50,)
# articles = pd.read_csv(NEWS, index_col=0)
# articles = pd.read_csv(NEWS, index_col=0, nrows=30000, skiprows=range(1, 150000))

# for i, row in tqdm(comp_occ.iterrows(), total=len(comp_occ)):
#     assert row.match_text == articles.loc[row.article_id].content[row.start_idx:row.end_idx], i

time: 268 ms


#### Outlier Related Investigation

In [95]:
matches = comp_occ[comp_occ.stock_symbol == 'REGN']
matches[matches.article_id.isin(articles.index)]
# .join(temp.reset_index().rename({'index': 'article_id'}, axis=1), lsuffix='l_')

,article_id,stock_symbol,match_text,start_idx,end_idx
263347,203653,REGN,Regeneron,705,714
263348,203653,REGN,Regeneron,1844,1853
263349,203653,REGN,Regeneron,2392,2401
264132,204912,REGN,Regeneron,3270,3279
264133,204912,REGN,Regeneron,3736,3745
264134,204912,REGN,Regeneron,3905,3914


time: 256 ms


In [ ]:
# print(temp.loc[204912].content)
print(temp.loc[203653].content)

#### Co-occurrences related Investigation

In [4]:
_, occ_per_article = nlp_utils.get_cooccurrences(comp_occ, debug=True)

def merge_ranges(index_pairs):
    ranges = []
    for x, y in index_pairs:
        x = max(x, 0)
        if len(ranges) == 0:
            ranges.append([x, y])
        else:
            _, last_y = ranges[-1]
            if x <= last_y and y > last_y:
                ranges[-1][1] = y
            elif x > last_y: 
                ranges.append([x, y])
    return ranges

def get_matching_article(symA, symB):
    x = occ_per_article[[symA, symB]]
    return x[(x[symA] != 0) & (x[symB] != 0)].index.tolist()

def print_cooccurrences(symA, symB):
    x = occ_per_article[[symA, symB]]
    for idx, row in x[(x[symA] != 0) & (x[symB] != 0)].iterrows():
        print(f'--->Article: {idx}<----', row[symA], row[symB])
        temp_occs = comp_occ[(comp_occ.article_id == idx) & (comp_occ.stock_symbol.isin([symA, symB]))]
        # temp_occs.sort_values('start_idx', ascending=True, inplace=True)
        init_indices = [(row.start_idx - 100, row.end_idx + 100) for i, row in temp_occs.iterrows()]
        indices = merge_ranges(init_indices)
        if len(init_indices) == len(indices):
            continue
        for x, y in indices:
            print('\n>>>', articles.loc[idx].content[x:y], '>>>\n')

time: 19.4 s


In [ ]:
# match_articles = get_matching_article('CB', 'ALL')
match_articles = get_matching_article('IBM', 'MCD')
# 'ADP', 'XOM'
# print(articles.loc[match_articles[1]].content)
print_cooccurrences('IBM', 'MCD')

In [ ]:
merged[(merged['Same Industry'] == 0) & (merged.Cooccurrences > 5) & (merged.Residuals > 0.25)]

### Try New Distance Measure

In [280]:
comp_distances = nlp_utils.get_distances(comp_occ)
# comp_distances_train = nlp_utils.get_distances(comp_occ_train)
people_distances = nlp_utils.get_distances(people_occ)
# mixed_distances = nlp_utils.get_distances(mixed_occ)

time: 21min 29s


In [281]:
comp_distances2 = nlp_utils.get_cheap_distances(comp_occ)
# comp_distances_train2 = nlp_utils.get_distances(comp_occ_train)
people_distances2 = nlp_utils.get_cheap_distances(people_occ)
# mixed_distances2 = nlp_utils.get_cheap_distances(mixed_occ)

time: 11min 26s


### Correlate Features

In [14]:
print('Significance for Pearson\'s r:')
crits = math_utils.rho_critical_values(len(all_correlations), prec=4)
for p_value, pearson_r in crits.iteritems():
    print(f'> {p_value:6}: {pearson_r}')

((all_correlations > crits[0.0001]) | (all_correlations < -crits[0.0001])).sum() / len(all_correlations)

Significance for Pearson's r:
>    0.1: 0.005
>   0.05: 0.0059
>   0.01: 0.0078
>  0.001: 0.01
> 0.0001: 0.0118


Price            0.994183
Return           0.999991
Normalized       0.834916
Residuals        0.830155
Same Industry    0.110761
dtype: float64

time: 314 ms


In [285]:
def make_mergable(x):
    if len(x.shape) == 2:
        x = x.stack()
    return x[[a < b for a, b in x.index]]

# comp_coocc[comp_coocc > 0] -> only 15.000 samples but higher correlation
merged = pd.concat([comp_coocc.rename('Cooccurrences'),  # 429
#                     comp_coocc_train.rename('Cooccurrences - Train'),  # 429
                    people_coocc.rename('Cooccurrences - People'),  # 200
#                     people_coocc_train.rename('Cooccurrences - People - Train'),  # 200
                    # mixed_coocc.rename('Cooccurrences - Mixed'),  # 200
                    make_mergable(comp_distances).rename('Dist'),
#                     make_mergable(comp_distances_train).rename('Dist - Train'),
                    make_mergable(people_distances).rename('Dist - People'),
                    # make_mergable(mixed_distances).rename('Dist - Mixed'),  # 450
                    make_mergable(comp_distances2).rename('Dist2'),
#                     make_mergable(comp_distances_train2).rename('Dist2 - Train'),
                    make_mergable(people_distances2).rename('Dist2 - People'),
                    # make_mergable(mixed_distances2).rename('Dist2 - Mixed'),
                    *[all_correlations[col].abs() for col in all_correlations.columns]  # 466
                   ], axis=1).dropna(subset=['Cooccurrences', 'Price']).fillna(0)
merged.index = merged.index.remove_unused_levels()
merged['Cooccurrences - Add'] = merged['Cooccurrences'] + merged['Cooccurrences - People']
merged['Dist - Add'] = merged['Dist'] + merged['Dist - People']
merged['Dist2 - Add'] = merged['Dist2'] + merged['Dist2 - People']

time: 7.64 s


In [ ]:
print(len(all_correlations.index), len(all_correlations.index.levels[0]), len(all_correlations.index.levels[1]))
print(len(comp_coocc.index), len(comp_coocc.index.levels[0]), len(comp_coocc.index.levels[1]))
print(len(merged.index), len(merged.index.levels[0]), len(merged.index.levels[1]))

In [12]:
print(len(all_correlations.index), len(all_correlations.index.levels[0]), len(all_correlations.index.levels[1]))
print(len(comp_coocc.index), len(comp_coocc.index.levels[0]), len(comp_coocc.index.levels[1]))
print(len(merged.index), len(merged.index.levels[0]), len(merged.index.levels[1]))

108811 466 466
97903 442 442
86736 416 416
time: 228 ms


In [286]:
merged.corr().round(4)

,Cooccurrences,Cooccurrences - People,Dist,Dist - People,Dist2,Dist2 - People,Price,Return,Normalized,Residuals,Same Industry,Cooccurrences - Add,Dist - Add,Dist2 - Add
Cooccurrences,1.0000,0.5239,0.1319,0.2279,0.1314,0.2183,0.0003,0.0640,0.0909,0.0945,0.0690,0.9999,0.1635,0.1594
Cooccurrences - People,0.5239,1.0000,0.0415,0.4456,0.0404,0.4648,0.0028,0.0273,0.0423,0.0452,0.0393,0.5345,0.1081,0.1049
Dist,0.1319,0.0415,1.0000,0.0720,0.8595,0.0757,-0.0037,0.1098,0.1261,0.1284,0.1624,0.1315,0.9885,0.8524
Dist - People,0.2279,0.4456,0.0720,1.0000,0.0682,0.8835,0.0055,0.0196,0.0375,0.0392,0.0362,0.2327,0.2219,0.1911
Dist2,0.1314,0.0404,0.8595,0.0682,1.0000,0.0767,-0.0240,0.0909,0.1106,0.1124,0.1447,0.1310,0.8506,0.9901
Dist2 - People,0.2183,0.4648,0.0757,0.8835,0.0767,1.0000,0.0071,0.0221,0.0369,0.0382,0.0424,0.2234,0.2080,0.2157
Price,0.0003,0.0028,-0.0037,0.0055,-0.0240,0.0071,1.0000,0.1588,0.0827,0.0810,0.0601,0.0003,-0.0028,-0.0225
Return,0.0640,0.0273,0.1098,0.0196,0.0909,0.0221,0.1588,1.0000,0.1715,0.1673,0.2868,0.0639,0.1103,0.0922
Normalized,0.0909,0.0423,0.1261,0.0375,0.1106,0.0369,0.0827,0.1715,1.0000,0.9866,0.2323,0.0909,0.1289,0.1135
Residuals,0.0945,0.0452,0.1284,0.0392,0.1124,0.0382,0.0810,0.1673,0.9866,1.0000,0.2391,0.0944,0.1314,0.1155


time: 340 ms


In [13]:
merged.corr().round(4)

,Cooccurrences,Cooccurrences - People,Dist,Dist - People,Dist2,Dist2 - People,Price,Return,Normalized,Residuals,Same Industry,Cooccurrences - Add,Dist - Add,Dist2 - Add
Cooccurrences,1.0000,0.5239,0.1319,0.2279,0.1314,0.2183,0.0003,0.0640,0.0909,0.0945,0.0690,0.9999,0.1635,0.1594
Cooccurrences - People,0.5239,1.0000,0.0415,0.4456,0.0404,0.4648,0.0028,0.0273,0.0423,0.0452,0.0393,0.5345,0.1081,0.1049
Dist,0.1319,0.0415,1.0000,0.0720,0.8595,0.0757,-0.0037,0.1098,0.1261,0.1284,0.1624,0.1315,0.9885,0.8524
Dist - People,0.2279,0.4456,0.0720,1.0000,0.0682,0.8835,0.0055,0.0196,0.0375,0.0392,0.0362,0.2327,0.2219,0.1911
Dist2,0.1314,0.0404,0.8595,0.0682,1.0000,0.0767,-0.0240,0.0909,0.1106,0.1124,0.1447,0.1310,0.8506,0.9901
Dist2 - People,0.2183,0.4648,0.0757,0.8835,0.0767,1.0000,0.0071,0.0221,0.0369,0.0382,0.0424,0.2234,0.2080,0.2157
Price,0.0003,0.0028,-0.0037,0.0055,-0.0240,0.0071,1.0000,0.1588,0.0827,0.0810,0.0601,0.0003,-0.0028,-0.0225
Return,0.0640,0.0273,0.1098,0.0196,0.0909,0.0221,0.1588,1.0000,0.1715,0.1673,0.2868,0.0639,0.1103,0.0922
Normalized,0.0909,0.0423,0.1261,0.0375,0.1106,0.0369,0.0827,0.1715,1.0000,0.9866,0.2323,0.0909,0.1289,0.1135
Residuals,0.0945,0.0452,0.1284,0.0392,0.1124,0.0382,0.0810,0.1673,0.9866,1.0000,0.2391,0.0944,0.1314,0.1155


time: 325 ms


In [16]:
merged.corr(method='kendall').round(4)

,Cooccurrences,Cooccurrences - People,Dist,Dist - People,Dist2,Dist2 - People,Price,Return,Normalized,Residuals,Same Industry,Cooccurrences - Add,Dist - Add,Dist2 - Add
Cooccurrences,1.0000,0.0880,0.9383,0.0879,0.9141,0.0879,-0.0245,0.0601,0.0623,0.0634,0.1431,0.9957,0.9349,0.9118
Cooccurrences - People,0.0880,1.0000,0.0810,0.9990,0.0773,0.9985,0.0056,0.0164,0.0164,0.0167,0.0456,0.1380,0.1352,0.1383
Dist,0.9383,0.0810,1.0000,0.0810,0.9591,0.0810,-0.0217,0.0590,0.0612,0.0622,0.1426,0.9345,0.9949,0.9546
Dist - People,0.0879,0.9990,0.0810,1.0000,0.0773,0.9993,0.0056,0.0164,0.0164,0.0167,0.0456,0.1379,0.1353,0.1384
Dist2,0.9141,0.0773,0.9591,0.0773,1.0000,0.0773,-0.0223,0.0556,0.0590,0.0600,0.1366,0.9103,0.9546,0.9937
Dist2 - People,0.0879,0.9985,0.0810,0.9993,0.0773,1.0000,0.0056,0.0164,0.0164,0.0167,0.0456,0.1379,0.1352,0.1384
Price,-0.0245,0.0056,-0.0217,0.0056,-0.0223,0.0056,1.0000,0.0989,0.0418,0.0410,0.0534,-0.0236,-0.0208,-0.0215
Return,0.0601,0.0164,0.0590,0.0164,0.0556,0.0164,0.0989,1.0000,0.0728,0.0687,0.1951,0.0605,0.0594,0.0560
Normalized,0.0623,0.0164,0.0612,0.0164,0.0590,0.0164,0.0418,0.0728,1.0000,0.8567,0.1410,0.0623,0.0612,0.0592
Residuals,0.0634,0.0167,0.0622,0.0167,0.0600,0.0167,0.0410,0.0687,0.8567,1.0000,0.1438,0.0634,0.0623,0.0602


time: 2.01 s


#### News Correlations

In [14]:
columns = ['Residuals', 'Cooccurrences', 'Dist2 - Add', 'Dist - Add', 'Same Industry']
renamed_columns = ['Stock Correlation', 'Co-occurrences', 'Minimum Distance', 'Pairwise Distance', 'Same Industry']
final = merged[columns].rename(dict(zip(columns, renamed_columns)), axis=1)
final.corr().round(4)

,Stock Correlation,Co-occurrences,Minimum Distance,Pairwise Distance,Same Industry
Stock Correlation,1.0000,0.0945,0.1155,0.1314,0.2391
Co-occurrences,0.0945,1.0000,0.1594,0.1635,0.0690
Minimum Distance,0.1155,0.1594,1.0000,0.8622,0.1477
Pairwise Distance,0.1314,0.1635,0.8622,1.0000,0.1643
Same Industry,0.2391,0.0690,0.1477,0.1643,1.0000


time: 282 ms


In [15]:
final.corr(method='kendall').round(4)

,Stock Correlation,Co-occurrences,Minimum Distance,Pairwise Distance,Same Industry
Stock Correlation,1.0000,0.0634,0.0602,0.0623,0.1438
Co-occurrences,0.0634,1.0000,0.9118,0.9349,0.1431
Minimum Distance,0.0602,0.9118,1.0000,0.9591,0.1377
Pairwise Distance,0.0623,0.9349,0.9591,1.0000,0.1435
Same Industry,0.1438,0.1431,0.1377,0.1435,1.0000


time: 388 ms


#### Stock Correlations

In [17]:
math_utils.r_critical_values(len(merged))

0.1000    0.01
0.0500    0.01
0.0100    0.01
0.0010    0.01
0.0001    0.01
dtype: float64

time: 192 ms


In [17]:
x = merged[['Dist - Add', *all_correlations.columns]].rename({'Dist - Add': 'Pairwise Distance'}, axis=1).corr().round(4)
x

,Pairwise Distance,Price,Return,Normalized,Residuals,Same Industry
Pairwise Distance,1.0000,-0.0028,0.1103,0.1289,0.1314,0.1643
Price,-0.0028,1.0000,0.1588,0.0827,0.0810,0.0601
Return,0.1103,0.1588,1.0000,0.1715,0.1673,0.2868
Normalized,0.1289,0.0827,0.1715,1.0000,0.9866,0.2323
Residuals,0.1314,0.0810,0.1673,0.9866,1.0000,0.2391
Same Industry,0.1643,0.0601,0.2868,0.2323,0.2391,1.0000


time: 261 ms


In [18]:
x = merged[['Dist - Add', *all_correlations.columns]].rename({'Dist - Add': 'Pairwise Distance'}, axis=1).corr(method='kendall').round(4)
x

,Pairwise Distance,Price,Return,Normalized,Residuals,Same Industry
Pairwise Distance,1.0000,-0.0208,0.0594,0.0612,0.0623,0.1435
Price,-0.0208,1.0000,0.0989,0.0418,0.0410,0.0534
Return,0.0594,0.0989,1.0000,0.0728,0.0687,0.1951
Normalized,0.0612,0.0418,0.0728,1.0000,0.8567,0.1410
Residuals,0.0623,0.0410,0.0687,0.8567,1.0000,0.1438
Same Industry,0.1435,0.0534,0.1951,0.1410,0.1438,1.0000


time: 627 ms


#### Final

In [19]:
print('\nPearson:\n')
print(utils.pandas_df_to_markdown_table(final.corr().round(4)))
print('\nSpearman:\n')
print(utils.pandas_df_to_markdown_table(final.corr(method='spearman').round(4)))
print('\nKendall:\n')
print(utils.pandas_df_to_markdown_table(final.corr(method='kendall').round(4)))


Pearson:

|        -          |   Stock Correlation |   Co-occurrences |   Minimum Distance |   Pairwise Distance |   Same Industry |
|:------------------|--------------------:|-----------------:|-------------------:|--------------------:|----------------:|
| Stock Correlation |              1      |           0.0945 |             0.1155 |              0.1314 |          0.2391 |
| Co-occurrences    |              0.0945 |           1      |             0.1594 |              0.1635 |          0.069  |
| Minimum Distance  |              0.1155 |           0.1594 |             1      |              0.8622 |          0.1477 |
| Pairwise Distance |              0.1314 |           0.1635 |             0.8622 |              1      |          0.1643 |
| Same Industry     |              0.2391 |           0.069  |             0.1477 |              0.1643 |          1      |

Spearman:

|        -          |   Stock Correlation |   Co-occurrences |   Minimum Distance |   Pairwise Distance |   S

Pearson:

|        -          |   Stock Correlation |   Co-occurrences |   Minimum Distance |   Pairwise Distance |   Same Industry |
|:------------------|--------------------:|-----------------:|-------------------:|--------------------:|----------------:|
| Stock Correlation |              1      |           0.0945 |             0.1155 |              0.1314 |          0.2391 |
| Co-occurrences    |              0.0945 |           1      |             0.1594 |              0.1635 |          0.069  |
| Minimum Distance  |              0.1155 |           0.1594 |             1      |              0.8622 |          0.1477 |
| Pairwise Distance |              0.1314 |           0.1635 |             0.8622 |              1      |          0.1643 |
| Same Industry     |              0.2391 |           0.069  |             0.1477 |              0.1643 |          1      |

Spearman:

|        -          |   Stock Correlation |   Co-occurrences |   Minimum Distance |   Pairwise Distance |   Same Industry |
|:------------------|--------------------:|-----------------:|-------------------:|--------------------:|----------------:|
| Stock Correlation |              1      |           0.08   |             0.077  |              0.0792 |          0.1761 |
| Co-occurrences    |              0.08   |           1      |             0.9846 |              0.9881 |          0.1477 |
| Minimum Distance  |              0.077  |           0.9846 |             1      |              0.9965 |          0.144  |
| Pairwise Distance |              0.0792 |           0.9881 |             0.9965 |              1      |          0.1493 |
| Same Industry     |              0.1761 |           0.1477 |             0.144  |              0.1493 |          1      |

Kendall:

|        -          |   Stock Correlation |   Co-occurrences |   Minimum Distance |   Pairwise Distance |   Same Industry |
|:------------------|--------------------:|-----------------:|-------------------:|--------------------:|----------------:|
| Stock Correlation |              1      |           0.0634 |             0.0602 |              0.0623 |          0.1438 |
| Co-occurrences    |              0.0634 |           1      |             0.9118 |              0.9349 |          0.1431 |
| Minimum Distance  |              0.0602 |           0.9118 |             1      |              0.9591 |          0.1377 |
| Pairwise Distance |              0.0623 |           0.9349 |             0.9591 |              1      |          0.1435 |
| Same Industry     |              0.1438 |           0.1431 |             0.1377 |              0.1435 |          1      |

- Normalization partly removed correlation between stock return and industry
    - Return vs. Same Industry: 0.2717 --> Normalized vs. Same Industry: 0.1960
- Correlation between Stock and Cooc increased with each preprocessing step
- "Cooc" / "Add Cooc" correlates with "Residuals"
- Addition of People Dist and Comp Dist increased Correlation
- Spearman: Higher for intra-text features (Cooc, Distances, Same Industry) --> monotonic, non-linear relationship, Lower in comparison with stock correlation --> linear relationship between both feature types
- Kendall: All values are lower than spearman, same as spearman compared to pearson
- Pearson/Spearman Significance: 0.10 -> 0.0058, 0.05 -> 0.0069, 0.01 -> 0.0090
- Kendall Significance: 0.10 -> 0.0038, 0.05 -> 0.0046, 0.01 -> 0.0060

Plot/Print diagonal:
- https://seaborn.pydata.org/examples/many_pairwise_correlations.html

#### Export

In [20]:
merged['Dist'].to_csv('../data/preprocessed/occurrences/pairwise_dist_2.csv')
merged['Dist - Add'].to_csv('../data/preprocessed/occurrences/pairwise_dist_with_people_2.csv')

c:\users\thomas\hpi\ma\venv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
c:\users\thomas\hpi\ma\venv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


time: 844 ms


### Show Case

In [79]:
NEWS = os.path.join("..", "data", "preprocessed", "news-v3.csv")
articles = pd.read_csv(NEWS, index_col=0, nrows=50000)  # , usecols=[0, 1, 2, 3]
articles.date = pd.to_datetime(articles.date)

time: 2.47 s


In [81]:
comp_occ = pd.read_csv('../data/preprocessed/occurrences/occurrences.csv', index_col=0).reset_index(drop=True)

time: 678 ms


In [76]:
x = comp_occ[comp_occ.article_id < 1000]
g = x.groupby(x.article_id)

time: 194 ms


In [77]:
g.count().stock_symbol.nlargest(10)

article_id
468    24
323    23
359    23
760    22
868    21
324    19
952    19
689    18
457    17
847    17
Name: stock_symbol, dtype: int64

time: 191 ms


#### Show Case for Defensis Slides

In [89]:
print(articles.loc[468].content[:504])

-- FedEx cancels Airbus A380 order, switches to Boeing
-- By  Paritosh Bansal
-- Tue Nov 7, 2006 3:58pm EST
-- http://www.reuters.com/article/2006/11/07/us-transport-fedex-boeing-idUSWNAS222320061107

 

 NEW YORK  (Reuters) - European plane maker Airbus ( EAD.PA ), which has faced a series of production delays on its high-profile A380 superjumbo program, suffered a major blow on Tuesday when FedEx Corp. ( FDX.N ) canceled a roughly $2.5 billion order and switched to rival Boeing Co. ( BA.N )
time: 287 ms


In [85]:
displacy.render({
        'text': articles.loc[468].content[212:504],  # .replace('$', '\$')
        'ents': [],
        'title': None
    }, style='ent', manual=True, jupyter=True)

time: 208 ms


In [87]:
displacy.render(nlp(articles.loc[468].content[212:504]), jupyter=True, style='ent')

time: 273 ms


In [86]:
def display_article(idx):
    occs = comp_occ
    hits = [{'start': match.start_idx - 212, 'end': match.end_idx - 212,
             'label': f'ORG ({match.stock_symbol})'}
            for _, match in occs[occs.article_id == idx].iterrows() if match.start_idx > 212 and match.end_idx < 504 ]
    ent_names = np.unique([x['label'] for x in hits])
    options = {'ents': ent_names, 'colors': plot.get_colors(ent_names), 'collapse_punct': True}
    displacy.render({
        'text': articles.loc[idx].content[212:504],  # .replace('$', '\$')
        'ents': hits,
        'title': None
    }, options=options, style='ent', manual=True, jupyter=True)

display_article(468)

time: 319 ms


#### Inspect european Debt Crisis

In [30]:
# for x in articles[articles.date == pd.to_datetime("2011-08-08")].filename.values:
#     print(x)

# for x in crisis_titles:
#     print(articles[articles.filename == x].index.values.min())

crisis_titles = [
  "vix-global-volatility-gauges-soar-after-s-p-lowers-u-s-rating",
  "us-eurozone-germany-idUSTRE7771QP20110808",
  "yen-strengthens-on-concern-u-s-slowdown-europe-s-debt-crisis-may-deepen",
  "yen-franc-gain-as-havens-on-u-s-slowdown-europe-debt-crisis-concerns",
  "volatility-climbs-from-asia-to-europe-after-s-p-s-u-s-downgrade",
  "basci-says-more-turkish-rate-cuts-possible-if-europe-can-t-end-debt-crisis",
  "european-u-s-stock-futures-tumble-on-u-s-debt-downgrade-rio-may-drop",
  "european-stocks-rise-as-ecb-said-to-buy-debt-u-s-index-futures-pare-loss",
  "european-stocks-halt-selloff-as-ecb-buys-bonds-santander-surges",
  "european-stocks-fall-21-from-high-as-banks-mining-shares-drop",
  "european-regulators-won-t-follow-greece-s-ban-on-short-selling",
  "european-coal-may-average-123-to-124-a-ton-barclays-says",
  "european-coal-advances-for-a-second-day-trades-at-127-20-a-ton",
  "european-central-bank-said-to-be-buying-italian-spanish-government-bonds",
  "europe-investor-confidence-plunges-most-on-record-on-debt-crises",
  "europe-gasoline-crack-down-shell-bp-sell-barges-oil-products",
  "euro-crisis-may-pack-u-s-banks-with-deposits-they-can-t-deploy",
  "eu-crisis-fix-lies-in-deployment-of-tools-not-more-summits-katainen-says",
]

crisis_ids = [
  251555,
  66822,
  251565,
  251564,
  251556,
  251136,
  251267,
  251266,
  251265,
  251264,
  251263,
  251262,
  251261,
  251260,
  251259,
  251258,
  251257,
  251256,
]

# x = articles.reset_index()
# x[x.date == pd.to_datetime("2011-08-08")].index.min(), x[x.date == pd.to_datetime("2011-08-08")].index.max()
# -> (211345, 211925)

time: 202 ms


In [76]:
NEWS = os.path.join("..", "data", "preprocessed", "news-v3.csv")
articles = pd.read_csv(NEWS, index_col=0, skiprows=itertools.chain(range(1, 211344), range(211926, int(1e7))))  # , usecols=[0, 1, 2, 3]
articles.date = pd.to_datetime(articles.date)

time: 6.38 s


In [77]:
for i, row in articles.loc[crisis_ids].iterrows():
    print("-"*50)
    print(i, row.filename)
    print(row.content[:1000])
    print("-"*50)

--------------------------------------------------
251555 vix-global-volatility-gauges-soar-after-s-p-lowers-u-s-rating
-- 
VIX, Global Volatility Gauges Soar After S&P Lowers U.S. Rating

-- B y   C e c i l e   V a n n u c c i ,   J e f f   K e a r n s   a n d   L y n n   T h o m a s s o n
-- 
2011-08-08T13:45:46Z

-- http://www.bloomberg.com/news/2011-08-08/vix-global-volatility-gauges-soar-after-s-p-lowers-u-s-rating.html
Volatility indexes soared around the
world, lifting the VIX for a third day and sending Europe’s
gauge toward a record ninth-straight gain, after  Standard &
Poor’s  stripped the U.S. of its top  credit rating .  The VIX, as the Chicago Board Options Exchange Volatility
Index is known, advanced 11 percent to 35.45 at 9:42 a.m. in New
York. Europe’s VStoxx Index climbed 8.7 percent to 42.68 and
Hong Kong’s HSI Volatility Index advanced 6.1 percent to 35.93,
both rising to the highest since May 2010.  Korea ’s Kospi 200
Volatility Index surged as much as 59 percent f

 sjones35@bloomberg.net  
--------------------------------------------------
--------------------------------------------------
251265 european-stocks-halt-selloff-as-ecb-buys-bonds-santander-surges
-- 
European Equities Decline for Seventh Day, Entering Bear Market

-- B y   S a r a h   J o n e s
-- 
2011-08-08T16:15:34Z

-- http://www.bloomberg.com/news/2011-08-08/european-stocks-halt-selloff-as-ecb-buys-bonds-santander-surges.html
European stocks dropped for a seventh
day to their lowest level since August 2009 as  Standard & Poor’s 
downgrade of U.S. sovereign debt overshadowed the  European
Central Bank ’s purchase of Spanish and Italian  government bonds .  Mining companies, carmakers and technology firms paced the
selloff amid concern that the lowering of America’s credit
rating will exacerbate the economic slowdown. Spanish lenders
retreated even after the ECB was said to buy the country’s
government bonds in a bid to tame the region’s debt crisis.  The benchmark Stoxx Europe 6

The four-week average for demand is also falling.  Gasoline for immediate loading in northw
--------------------------------------------------
--------------------------------------------------
251257 euro-crisis-may-pack-u-s-banks-with-deposits-they-can-t-deploy
-- 
Euro Crisis May Pack U.S. Banks With Deposits They Can’t Use

-- B y   B r a d l e y   K e o u n   a n d   D a k i n   C a m p b e l l
-- 
2011-08-08T15:55:35Z

-- http://www.bloomberg.com/news/2011-08-08/euro-crisis-may-pack-u-s-banks-with-deposits-they-can-t-deploy.html
The European debt crisis is poised to
flood U.S. banks with something they don’t want and can’t use:
more money.  Cash held by U.S. banks surged 8.4 percent to a record $981
billion during the week ending July 27, the  Federal Reserve  said
in an Aug. 5 report. That’s more than triple the amount firms
had in July 2008, before the collapse of Lehman Brothers
Holdings Inc. almost froze bank-to-bank lending.  Even more money may be deposited with U.S. lender